In [3]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from statsmodels.stats.anova import anova_lm

In [4]:
from pygam import s as s_gam, l as l_gam, f as f_gam, LinearGAM, LogisticGAM
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit, cross_validate
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, KBinsDiscretizer, SplineTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import StandardScaler


In [5]:
Wage = load_data("Wage")

Wage

,year,age,maritl,race,education,region,jobclass,health,health_ins,logwage,wage
0,2006,18,1. Never Married,1. White,1. < HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,2. No,4.318063,75.043154
1,2004,24,1. Never Married,1. White,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,2. No,4.255273,70.476020
2,2003,45,2. Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,1. <=Good,1. Yes,4.875061,130.982177
3,2003,43,2. Married,3. Asian,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,1. Yes,5.041393,154.685293
4,2005,50,4. Divorced,1. White,2. HS Grad,2. Middle Atlantic,2. Information,1. <=Good,1. Yes,4.318063,75.043154
...,...,...,...,...,...,...,...,...,...,...,...
2995,2008,44,2. Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,2. >=Very Good,1. Yes,5.041393,154.685293
2996,2007,30,2. Married,1. White,2. HS Grad,2. Middle Atlantic,1. Industrial,2. >=Very Good,2. No,4.602060,99.689464
2997,2005,27,2. Married,2. Black,1. < HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,2. No,4.193125,66.229408
2998,2005,27,1. Never Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,2. >=Very Good,1. Yes,4.477121,87.981033


In [6]:
def clean_text_series(series):
    return (
        series.str.lower()
        .str.replace(r"^\d+\.\s*", "", regex=True)
        .str.replace(" ", "_")
    )


wage_df = Wage.copy()

wage_df_categorical_cols = wage_df.select_dtypes(include=["object", "category"]).columns

wage_df[wage_df_categorical_cols] = (
    wage_df.select_dtypes(include=["object", "category"]).apply(clean_text_series)
)

wage_df

,year,age,maritl,race,education,region,jobclass,health,health_ins,logwage,wage
0,2006,18,never_married,white,<_hs_grad,middle_atlantic,industrial,<=good,no,4.318063,75.043154
1,2004,24,never_married,white,college_grad,middle_atlantic,information,>=very_good,no,4.255273,70.476020
2,2003,45,married,white,some_college,middle_atlantic,industrial,<=good,yes,4.875061,130.982177
3,2003,43,married,asian,college_grad,middle_atlantic,information,>=very_good,yes,5.041393,154.685293
4,2005,50,divorced,white,hs_grad,middle_atlantic,information,<=good,yes,4.318063,75.043154
...,...,...,...,...,...,...,...,...,...,...,...
2995,2008,44,married,white,some_college,middle_atlantic,industrial,>=very_good,yes,5.041393,154.685293
2996,2007,30,married,white,hs_grad,middle_atlantic,industrial,>=very_good,no,4.602060,99.689464
2997,2005,27,married,black,<_hs_grad,middle_atlantic,industrial,<=good,no,4.193125,66.229408
2998,2005,27,never_married,white,some_college,middle_atlantic,industrial,>=very_good,yes,4.477121,87.981033


In [7]:
y = wage_df["wage"]
X = wage_df.drop(["wage", "logwage"], axis=1)

In [8]:
cols_preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="if_binary", sparse_output=False), wage_df_categorical_cols),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
    
).set_output(transform="pandas")

### Polynomial Regression

In [9]:
cols_poly = ColumnTransformer(
    transformers=[
        ("poly_age", PolynomialFeatures(include_bias=False), ["age"]),
        ("poly_year", PolynomialFeatures(include_bias=False), ["year"]),
    ],
    remainder="passthrough",
    verbose_feature_names_out=True,
    
)

In [10]:
poly_pipe = Pipeline(
    steps=[
        ("cols_preprocess", cols_preprocess),
        ("cols_poly", cols_poly),
        ("scaler", StandardScaler()),
        ("ols", LinearRegression()),
    ]
)


param_grid = {
    "cols_poly__poly_age__degree": range(1, 5),
    "cols_poly__poly_year__degree": range(1, 5),
}

poly_grid = GridSearchCV(
    estimator=poly_pipe,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
)

poly_results = poly_grid.fit(X, y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [11]:
poly_results.best_params_

{'cols_poly__poly_age__degree': 3, 'cols_poly__poly_year__degree': 1}

In [12]:
poly_results.best_score_

-1144.1453906683369

### Splines

In [13]:
cols_spline = ColumnTransformer(
    transformers=[
        (
            "spline_age",
            SplineTransformer(
                n_knots=5, degree=3, include_bias=False, knots="quantile"
            ),
            ["age"],
        ),
        (
            "spline_year",
            SplineTransformer(include_bias=False, knots="quantile"),
            ["year"],
        ),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")

In [14]:
spline_pipe = Pipeline(
    steps=[
        ("cols_preprocess", cols_preprocess),
        ("cols_spline", cols_spline),
        ("scaler", StandardScaler()),
        ("ols", LinearRegression()),
    ]
)


param_grid = {
    "cols_spline__spline_age__degree": range(1, 3),
    "cols_spline__spline_age__n_knots": range(1, 10),
    "cols_spline__spline_year__degree": range(1, 3),
    "cols_spline__spline_year__n_knots": range(1, 10),
}

spline_grid = GridSearchCV(
    estimator=spline_pipe,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=1,
)

spline_results = spline_grid.fit(X, y)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


c:\Users\PC\Documents\envs\islp_py310\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
340 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
340 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\PC\Documents\envs\islp_py310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\PC\Documents\envs\islp_py310\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\PC\Documents\envs\islp_py310\lib\site-packages\sklearn\pipeline.py", line 423, in fit
    Xt = self._fit(X, y

In [15]:
spline_results.best_params_

{'cols_spline__spline_age__degree': 1,
 'cols_spline__spline_age__n_knots': 3,
 'cols_spline__spline_year__degree': 2,
 'cols_spline__spline_year__n_knots': 2}

In [16]:
spline_results.best_score_

-1143.610464277837

In [21]:
spline_estimator = spline_results.best_estimator_